In [4]:
import numpy as np

def perform_svd_analysis(input_matrix):
    """
    Decomposes a matrix using Singular Value Decomposition (SVD) and computes related properties:
    the pseudo-inverse, condition number, and reconstructed matrix.

    Parameters:
    ----------
    input_matrix : np.ndarray
        A 2D NumPy array to be decomposed.

    Returns:
    -------
    left_vectors : np.ndarray
        Left singular vectors (U matrix) of the input matrix.
    singular_values_diag : np.ndarray
        Diagonal matrix containing singular values.
    right_vectors_transpose : np.ndarray
        Right singular vectors (V transpose) of the input matrix.
    cond_number : float
        Condition number, determined as the ratio of the largest to smallest singular values.
    pseudo_inverse : np.ndarray
        Pseudo-inverse of the input matrix.

    Raises:
    ------
    ValueError:
        If the matrix is singular and does not have an inverse.
    """

    # Step 1: Compute transpose_product = A^T * A to derive right singular vectors
    transpose_product = np.dot(input_matrix.T, input_matrix)
    
    # Step 2: Perform eigen decomposition to find right singular vectors and eigenvalues
    eigenvalues, right_vectors = np.linalg.eig(transpose_product)
    
    # Step 3: Sort eigenvalues and corresponding eigenvectors in descending order
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_indices]
    right_vectors = right_vectors[:, sorted_indices]
    
    # Step 4: Compute singular values (square root of eigenvalues)
    singular_values = np.sqrt(eigenvalues)
    if singular_values[-1] == 0:
        raise ValueError("Matrix is singular and does not have an inverse.")
    
    # Step 5: Calculate left singular vectors using U = A * V / singular_values
    left_vectors = input_matrix @ right_vectors / singular_values
    left_vectors /= np.linalg.norm(left_vectors, axis=0)  # Normalize columns of U

    # Step 6: Create a diagonal matrix of singular values
    singular_values_diag = np.diag(singular_values)
    
    # Step 7: Compute the pseudo-inverse of singular values matrix
    singular_values_inverse = np.diag(1 / singular_values)
    
    # Step 8: Compute the pseudo-inverse of the input matrix
    pseudo_inverse = right_vectors @ singular_values_inverse @ left_vectors.T

    # Step 9: Calculate the condition number (ratio of max to min singular values)
    cond_number = singular_values[0] / singular_values[-1]

    return left_vectors, singular_values_diag, right_vectors.T, cond_number, pseudo_inverse

# Example matrix for testing
test_matrix = np.array([[1, 2, 1], [4, 3, 7], [7, 9, 2]], dtype=float)

# Apply the custom SVD function
try:
    left_vecs, singular_vals_diag, right_vecs_t, cond_num, pseudo_inv = perform_svd_analysis(test_matrix)

    # Display results from custom implementation
    print("Custom SVD U (Left Singular Vectors):\n", left_vecs)
    print("Custom SVD S (Singular Values Diagonal):\n", singular_vals_diag)
    print("Custom SVD V^T (Right Singular Vectors Transpose):\n", right_vecs_t)
    print("Custom Condition Number:", cond_num)
    print("Custom Pseudo-Inverse:\n", pseudo_inv)

    # Reconstruct the matrix using custom SVD results
    reconstructed_matrix = left_vecs @ singular_vals_diag @ right_vecs_t
    print("Reconstructed Matrix (Custom SVD):\n", reconstructed_matrix)

except ValueError as error:
    print(error)

print("_________________________")

# Using NumPy's SVD for comparison
left_np, singular_vals_np, right_np_t = np.linalg.svd(test_matrix)
singular_vals_np_diag = np.diag(singular_vals_np)
pseudo_inverse_np = np.dot(right_np_t.T, np.dot(np.diag(1 / singular_vals_np), left_np.T))
cond_num_np = np.linalg.cond(test_matrix)

# Display results from NumPy's implementation
print("\nNumPy SVD U:\n", left_np)
print("NumPy SVD S (Singular Values Diagonal):\n", singular_vals_np_diag)
print("NumPy SVD V^T:\n", right_np_t)
print("NumPy Pseudo-Inverse:\n", pseudo_inverse_np)
print("NumPy Condition Number:\n", cond_num_np)

# Reconstruct the matrix using NumPy's SVD results
reconstructed_matrix_np = left_np @ singular_vals_np_diag @ right_np_t
print("Reconstructed Matrix (NumPy SVD):\n", reconstructed_matrix_np)


Custom SVD U (Left Singular Vectors):
 [[ 0.17552297 -0.01988568 -0.98427448]
 [ 0.53923918  0.83841813  0.07922216]
 [ 0.82365818 -0.54466467  0.15788477]]
Custom SVD S (Singular Values Diagonal):
 [[13.5987949   0.          0.        ]
 [ 0.          5.36396002  0.        ]
 [ 0.          0.          0.54837044]]
Custom SVD V^T (Right Singular Vectors Transpose):
 [[ 0.59550034  0.68989106  0.41161835]
 [-0.08927469 -0.45237082  0.88735036]
 [ 0.79837922 -0.56516454 -0.20779717]]
Custom Condition Number: 24.798555729624834
Custom Pseudo-Inverse:
 [[-1.425  0.125  0.275]
 [ 1.025 -0.125 -0.075]
 [ 0.375  0.125 -0.125]]
Reconstructed Matrix (Custom SVD):
 [[1. 2. 1.]
 [4. 3. 7.]
 [7. 9. 2.]]
_________________________

NumPy SVD U:
 [[-0.17552297  0.01988568 -0.98427448]
 [-0.53923918 -0.83841813  0.07922216]
 [-0.82365818  0.54466467  0.15788477]]
NumPy SVD S (Singular Values Diagonal):
 [[13.5987949   0.          0.        ]
 [ 0.          5.36396002  0.        ]
 [ 0.          0.    